In [2]:
# Grouping data by intervals

# 15 seconds interval, without region for partitioning
import pyspark.sql.functions as F

days_to_analyze = [1,4,12,17,20]

for day in days_to_analyze:
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/13-speed-calculation-filtered-columns/MO_1510"+str(day)+"/")
    df_graph_id = traces\
        .withColumn('minute_avl',F.minute(F.col("dt_avl")))\
        .withColumn("second_avl",F.second("dt_avl"))\
        .withColumn("graph_id",F.concat(F.col("hour_avl"),F.lit("-"),F.floor((F.col("minute_avl")*60+F.col("second_avl"))/15)))

    df_graph_id.repartition("graph_id").write.parquet("s3://mobility-traces-sp/processed-data-avl-date/14-speed-and-graph-id-15-seconds/MO_1510"+str(day)+"/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Grouping data by intervals

# 15 seconds interval, with region for partitioning
import pyspark.sql.functions as F

days_to_analyze = [1,4,12,17,20]

for day in days_to_analyze:
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/13-speed-calculation-filtered-columns/MO_1510"+str(day)+"/")
    df_graph_id = traces\
        .withColumn('minute_avl',F.minute(F.col("dt_avl")))\
        .withColumn("second_avl",F.second("dt_avl"))\
        .withColumn("graph_id",F.concat(F.col("hour_avl"),F.lit("-"),F.floor((F.col("minute_avl")*60+F.col("second_avl"))/15),F.lit("-"),F.col("region")))

    df_graph_id.repartition("graph_id").write.parquet("s3://mobility-traces-sp/processed-data-avl-date/14-speed-and-graph-id-15s-with-region/MO_1510"+str(day)+"/")

In [ ]:
# 15 seconds with region

import pyspark.sql.functions as f

new_df = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/14-speed-and-graph-id-15s-with-region/MO_1510"+str(1)+"/")
new_df = new_df.repartition(150)

df2 = new_df

new_df.alias('df1').join(df2.alias("df2"),on=["graph_id"],how="outer")\
    .select(
        f.col("df1.id_avl").alias("id_avl_1"),
        f.col("df1.line_id").alias("line_1"),
        f.col("df1.latitude").alias("latitude_1"),
        f.col("df1.longitude").alias("longitude_1"),
        f.col("df2.id_avl").alias("id_avl_2"),
        f.col("df2.line_id").alias("line_2"),
        f.col("df2.latitude").alias("latitude_2"),
        f.col("df2.longitude").alias("longitude_2"),
        f.col("graph_id"),
    
).write.parquet("s3://mobility-traces-sp/teste_MO_15101_15s_with-region/")


In [2]:
# 5 min interval with region

import pyspark.sql.functions as f

new_df = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/14-speed-and-graph-id/MO_1510"+str(1)+"/")
new_df = new_df.repartition(150)

new_df = new_df.withColumn('graph_id_novo',f.concat(f.col("hour_avl"),f.lit("-"),f.floor(f.col("minute_avl")/5),f.lit("-"),f.col("region")))

df2 = new_df

new_df.alias('df1').join(df2.alias("df2"),on=["graph_id_novo"],how="outer")\
    .select(
        f.col("df1.id_avl").alias("id_avl_1"),
        f.col("df1.line_id").alias("line_1"),
        f.col("df1.latitude").alias("latitude_1"),
        f.col("df1.longitude").alias("longitude_1"),
        f.col("df2.id_avl").alias("id_avl_2"),
        f.col("df2.line_id").alias("line_2"),
        f.col("df2.latitude").alias("latitude_2"),
        f.col("df2.longitude").alias("longitude_2"),
        f.col("graph_id_novo").alias("graph_id"),
    
).write.parquet("s3://mobility-traces-sp/teste_minute_MO_15101_5min/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# 1 min interval + region

import pyspark.sql.functions as f

new_df = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/14-speed-and-graph-id/MO_1510"+str(1)+"/")
new_df = new_df.repartition(150)

new_df = new_df.withColumn('graph_id',f.concat(f.col("hour_avl"),f.lit("-"),f.col("minute_avl"),f.lit("-"),f.col("region")))

df2 = new_df

new_df.alias('df1').join(df2.alias("df2"),on=["graph_id"],how="outer")\
    .select(
        f.col("df1.id_avl").alias("id_avl_1"),
        f.col("df1.line_id").alias("line_1"),
        f.col("df1.latitude").alias("latitude_1"),
        f.col("df1.longitude").alias("longitude_1"),
        f.col("df2.id_avl").alias("id_avl_2"),
        f.col("df2.line_id").alias("line_2"),
        f.col("df2.latitude").alias("latitude_2"),
        f.col("df2.longitude").alias("longitude_2"),
        f.col("graph_id").alias("graph_id"),
    
).write.parquet("s3://mobility-traces-sp/teste_minute_MO_15101/")